In [ ]:
!pip install mtcnn --q

In [ ]:
from mtcnn import MTCNN

In [ ]:
test_images_dir = '/content/extracted_frames/test'

In [ ]:
def load_image_paths(directory):
    image_paths = []
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            image_paths.append(os.path.join(directory, filename))
    return image_paths

In [ ]:
def detect_and_crop_faces(image_path, detector):
    image = Image.open(image_path)
    image_np = np.asarray(image)
    result = detector.detect_faces(image_np)
    if result:
        for person in result:
            bounding_box = person['box']
            keypoints = person['keypoints']

            # Crop the detected face
            x, y, width, height = bounding_box
            cropped_face = image_np[y:y+height, x:x+width]

            # Convert the cropped face back to an image
            cropped_face_image = Image.fromarray(cropped_face)

            return cropped_face_image
    return None

In [ ]:
# Initialize the MTCNN face detector
detector = MTCNN()

In [ ]:
# Directory containing the original images
test_image_paths = load_image_paths(test_images_dir)

# Directory to save cropped face images
output_dir = '/content/cropped_faces/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Process each image and save the cropped face with the original filename
for image_path in test_image_paths:
    cropped_face = detect_and_crop_faces(image_path, detector)
    # print("Image path is:", test_image_paths)
    if cropped_face:
        # Extract the original filename
        original_filename = os.path.basename(image_path)
        # Save the cropped face image with the original filename
        cropped_face.save(os.path.join(output_dir, original_filename))

In [ ]:
# Function to resize images in a directory
def resize_images(directory, target_size=(224, 224)):
    resized_images = []
    test_image_paths = []
    test_image_labels = []

    # Iterate over each image in the directory
    for filename in os.listdir(directory):
        if filename.endswith(".jpg") or filename.endswith(".png"):
            label = filename.split('.')[0].split('_')[0]
            image_path = os.path.join(directory, filename)
            image = cv2.imread(image_path)
            resized_image = cv2.resize(image, target_size)

            # Append resized image, image path, and label
            resized_images.append(resized_image)
            test_image_paths.append(image_path)
            test_image_labels.append(int(label))  # Convert label to int

    # Convert lists to numpy arrays
    resized_images = np.array(resized_images)
    test_image_labels = np.array(test_image_labels)

    return resized_images, test_image_paths, test_image_labels

In [ ]:
# Resize images in test directory
cropped_resized_images, cropped_test_image_paths, cropped_test_image_labels = resize_images(output_dir)

# Check the shape of resized images
print("Resized images shape:", cropped_resized_images.shape)

In [ ]:
# Convert labels to one-hot encoding
cropped_test_image_labels_onehot = to_categorical(cropped_test_image_labels, num_classes=2)

In [ ]:
# Evaluate the model
cropped_test_loss, cropped_test_accuracy = model.evaluate(cropped_resized_images, cropped_test_image_labels_onehot)

In [ ]:
# Predictions
cropped_predictions = model.predict(cropped_resized_images)
cropped_predicted_scores = cropped_predictions[:, 1]  # Assuming class 1 corresponds to index 1 in predictions

# Normalize scores between 0 and 1
cropped_predicted_scores_normalized = (cropped_predicted_scores - np.min(cropped_predicted_scores)) / (np.max(cropped_predicted_scores) - np.min(cropped_predicted_scores))

In [ ]:
# Save predictions to a CSV file
output_file = 'predictions.csv'
with open(output_file, 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow(['filename', 'liveness_score', 'liveness_score_crop'])  # Header
    for i, filename in enumerate(test_image_paths):
        writer.writerow([filename, predicted_scores_normalized[i], cropped_predicted_scores_normalized[i]])

print(f"Predictions saved to {output_file}")